In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Load data.

data_train = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
data_test = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")

In [ ]:
#Drop id from the data since we won't be using it to train the model and split the features and the target.

X = data_train.drop(['Id', 'Cover_Type'], axis = 1)
y = data_train.Cover_Type

In [ ]:
#Let's take a look at the target

y.unique()

In [ ]:
#Since our target seems to be a categorical feature with 7 unique values.
new_y = []
i = 0
for row in y:
    new_y.append([0]*7)
    new_y[i][row-1] = 1
    i = i+1
print("done")


In [ ]:
new_y_df = pd.DataFrame(new_y)

In [ ]:
#Split our data into data we will use to train and data we will use to evaluate our model.

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, new_y_df, random_state = 1, test_size = 0.2, train_size = 0.8)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
    layers.Dense(units = 500, activation = 'relu', input_shape = [54]),
    layers.Dense(units = 500, activation = 'relu'),
    layers.Dense(units = 500, activation = 'relu'),
    layers.Dense(units = 500, activation = 'relu'),
    layers.Dense(units = 7, activation = 'softmax'),
])

model.compile(
    optimizer = 'adam',
    loss = 'CategoricalCrossentropy',
)

print("did it work?")

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta = 0.0001,
    patience = 20,
    restore_best_weights = True
)

In [ ]:
#Fit the data into the model

model.fit(X_train, y_train,
          validation_data =(X_valid, y_valid),
          batch_size = 1024,
          epochs = 100,
          callbacks = [early_stopping]
         )

print("Is it done?")

In [ ]:
#Drop the id from test data and predict

X_test = data_test.drop(['Id'], axis = 1)
predictions = model.predict(X_test)


In [ ]:
#The predictions are in form of an array with the probability of each value be true, now we make it predict the most likely value
new_predictions = []
for values in predictions:
    _max = np.argmax(values)+1
    new_predictions.append(_max)
len(new_predictions)

In [ ]:
#Putting our predictions on competition format to submit

predictions_df = pd.DataFrame(new_predictions, columns = ['Cover_Type'])
index = data_test.Id
output = pd.concat([index, predictions_df], axis = 1)
output.to_csv('my_submission.csv', index=False)